In [1]:
def test_data(tttt):
    uttime = datetime.utcfromtimestamp(tttt).strftime('%Y-%m-%dT%H:%M:%S')
    X=api.exchange_rates_get_specific_rate('BTC', 'USD', {'time': uttime})
    return(X)               

In [ ]:
import asyncio

async def test(tttt):
    uttime = datetime.utcfromtimestamp(tttt).strftime('%Y-%m-%dT%H:%M:%S')
    X=api.exchange_rates_get_specific_rate('BTC', 'USD', {'time': uttime})
    print(X) 
# Busy waits for a second, but we don't want to stick around...
    #print('gr1 started work: {}'.format(tic()))
    #await asyncio.sleep(2)
    #print('gr1 ended work: {}'.format(tic()))
tttt=1529980664
ioloop = asyncio.get_event_loop()
tasks = [asyncio.ensure_future(test(tttt + 10*t)) for t in range(1)]
ioloop.run_until_complete(tasks)
ioloop.close()


In [ ]:
mp.cpu_count()
#import multiprocessing
#This code does not improve on sending naive sequential requests.
#
#tttt= 1529969400
#jobs=[]
#print(time.time(),'start')
#for i in range(10):
#    p = multiprocessing.Process(target= test_data(tttt+10*i))
#    jobs.append(p)
#    p.start()
#print(time.time(), 'end')

In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes')
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp

test_key = 'DB318A59-25FF-499E-9A6D-783A19C346D8'

#Free key for testing: test_key = 'EED0F746-36FB-4CC4-8E7C-527333DFA6FB'

api = CoinAPIv1(test_key)

#this function computes how many seconds from now till midnight.
#the correction is due to the fact that some times are not utc by default.
#This can be checked transforming output into utc time.
def until_midnight():
    tomorrow = datetime.now() + timedelta(1)
    midnight = datetime(year=tomorrow.year, month=tomorrow.month, 
                        day=tomorrow.day, hour=0, minute=0, second=0)
    wrong_number=(midnight - datetime.now()).seconds
    return (wrong_number + 61)# - 14399)

In [11]:
def request_rates(unix_time, base, quote): 
    path='/home/fbuonerba/exchange_rates_data/'
    utctime = datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%dT%H:%M:%S')
    while True:
        try:
            exchange=api.exchange_rates_get_specific_rate(base, quote, {'time': utctime})
            with open(path+'exchange_rate_'+str(base)+'_'+str(quote)+'_'+str(unix_time)+'.txt', 'w') as outfile:  
                json.dump(exchange, outfile) 
            break
        except urllib.error.HTTPError as e:
            print(e, unix_time)
            time.sleep(10)
    return(exchange)



print(time.time())
unix_time= 1930019800
pool = mp.Pool()#processes=7)

for i in range(3):
    print(time.time(), 'start round =', i)
    results = [pool.apply_async(request_rates, args=(unix_time + 600*t,'ETH','USD',)) for t in range(2)]
    output = [res.get() for res in results]
    print(output)
    print(time.time(), 'end round =', i)
    unix_time+=1200

1530099378.328853
1530099378.3665092 start round = 0
HTTP Error 550:  1930020400
HTTP Error 550:  1930019800
HTTP Error 550:  1930020400
HTTP Error 550:  1930019800
HTTP Error 550:  1930020400
HTTP Error 550:  1930019800


Process ForkPoolWorker-147:
Process ForkPoolWorker-144:
Process ForkPoolWorker-143:
Process ForkPoolWorker-145:
Process ForkPoolWorker-142:
Process ForkPoolWorker-141:
Process ForkPoolWorker-146:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-140:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "<ipython-input-11-29d2d5546f34>", line 6, in req

KeyboardInterrupt: 


During handling of the above exception, another exception occurred:

  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/urllib/request.py", line 510, in error
    return self._call_chain(*args)
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.5/urllib/request.py", line 444, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.5/urllib/request.py", line 590, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP Error 550: 
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)

During handling of the above exception, another exception occurred:

  File "/usr/lib/python3.5/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
  File "<ipython-input-11-29d2d5546f34>", line 12, in request_rates
    

In [ ]:
#class HelloWorld(luigi.Task):
#    def requires(self):
#        return None
#    def output(self):
#        return luigi.LocalTarget('helloworld.txt')
#    def run(self):
#        with self.output().open('w') as outfile:
#            outfile.write('Hello World!\n')

#class NameSubstituter(luigi.Task):
#    name = luigi.Parameter()
#    def requires(self):
#        return HelloWorld()
#    def output(self):
#        return luigi.LocalTarget(self.input().path + '.name_' + self.name)
#    def run(self):
#        with self.input().open() as infile, self.output().open('w') as outfile:
#            text = infile.read()
#            text = text.replace('World', self.name)
#            outfile.write(text)

In [ ]:
import luigi
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes')
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request

test_key = 'DB318A59-25FF-499E-9A6D-783A19C346D8'

#Free key for testing: test_key = 'EED0F746-36FB-4CC4-8E7C-527333DFA6FB'

api = CoinAPIv1(test_key)

#this function computes how many seconds from now till midnight.
#the correction is due to the fact that some times are not utc by default.
#This can be checked transforming output into utc time.
def until_midnight():
    tomorrow = datetime.now() + timedelta(1)
    midnight = datetime(year=tomorrow.year, month=tomorrow.month, 
                        day=tomorrow.day, hour=0, minute=0, second=0)
    wrong_number=(midnight - datetime.now()).seconds
    return (wrong_number + 61)# - 14399)


In [ ]:
#this task downloads data (by luigi default, if it does not exist already).
class request_exchange_rates(luigi.Task):
    unix_time=luigi.Parameter()
    base=luigi.Parameter()
    quote=luigi.Parameter()
    path='/home/fbuonerba/exchange_rates_data/'
    def requires(self):
        return None
    def output(self):
        return luigi.LocalTarget(self.path+'exchange_rate_'+str(self.base)+'_'+str(self.quote)+'_'+str(self.unix_time)+'.txt')
    def run(self):
        while True:
            try: 
                utctime = datetime.utcfromtimestamp(self.unix_time).strftime('%Y-%m-%dT%H:%M:%S')
                exchange=api.exchange_rates_get_specific_rate(self.base, self.quote, {'time': utctime})
                with self.output().open('w') as outfile:
                    json.dump(exchange, outfile) 
                print('okokokokokok', self.unix_time)
                break
            except urllib.error.HTTPError as err:
                print('AAAAAAAAAAAAAAAAAAAA', err, self.unix_time)
                time.sleep(30)
            
        
    

In [ ]:
unix_time= 1529985000
#1586918800  1529954000
while True:
    luigi.build([request_exchange_rates(unix_time + 600*t, base='ETH', quote='USD') for t in range(2)], workers=2, local_scheduler=True)
    unix_time+=1200
    #luigi.build([request_exchange_rates(unix_time + 600*t, base='ETH', quote='USD') for t in range(100)], workers=1, local_scheduler=True)
    